#### This notebook will be mainly used for the IBM Data Science Specialization capstone project.

In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import lxml
import html5lib 
import requests
import re

### Task for week 1:

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Task for week 3 (part 1)
#### Scraping data from Wikipedia page 
As the Wikipedia page was edited after scraping the page and completing the task, I am attaching a link from the Wikipedia page edit history as of 28 March 2020. This is an old revision of this page, as edited by 47.9.215.179 (talk) at 11:16, 28 March 2020 

In [3]:
# url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# This is an old revision of this page, as edited by 47.9.215.179 (talk) at 11:16, 28 March 2020 
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=947772202"
dframe = pd.read_html(url, header=None)[0]

In [4]:
dframe.head(5)

,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7ADowntown Toronto(Queen's Park / Ontario Pro...,M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...
2,M1CScarborough(Rouge Hill / Port Union / Highl...,M2CNot assigned,M3CNorth York(Don Mills)South(Flemingdon Park),M4CEast York(Woodbine Heights),M5CDowntown Toronto(St. James Town),M6CYork(Humewood-Cedarvale),M7CNot assigned,M8CNot assigned,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...
3,M1EScarborough(Guildwood / Morningside / West ...,M2ENot assigned,M3ENot assigned,M4EEast Toronto(The Beaches),M5EDowntown Toronto(Berczy Park),M6EYork(Caledonia-Fairbanks),M7ENot assigned,M8ENot assigned,M9ENot assigned
4,M1GScarborough(Woburn),M2GNot assigned,M3GNot assigned,M4GEast York(Leaside),M5GDowntown Toronto(Central Bay Street),M6GDowntown Toronto(Christie),M7GNot assigned,M8GNot assigned,M9GNot assigned


In [5]:
dframe.shape

(20, 9)

#### Appending the table data into a list:

In [6]:
lst = []

for row in range(dframe.shape[0]):
    for col in range(dframe.shape[1]):
        lst.append(dframe.iloc[row, col])

###### Example: Cell with two boroughs (before transformation)

In [7]:
print(dframe.iloc[7,2])
print(lst[9*7+2])

M3KNorth York(Downsview)East (CFB Toronto)
M3KNorth York(Downsview)East (CFB Toronto)


#### Transforming the imported table:

In [8]:
df=pd.DataFrame(np.zeros((len(lst), 3)))

# Entering PostalCode:
df[0]=[string[:3] for string in lst]

# Entering Borough data and removing Neighborhood data, also for cells with two Boroughs:
df[1]=[string[3:] for string in lst]
df[1]=[re.sub(r'\(.*\) ?(.*) ?\(.*\)', r', \1', string) for string in df[1]]
df[1]=[re.sub(r'\(.*\)', r'', string) for string in df[1]]

# Extracting Neighborhood data from string, also for cells with two Boroughs:
df[2]=[string[3:] for string in lst]
df[2]=[re.sub(r'.*\((.*)\).*\((.*)\)', r'\1, \2', string) for string in df[2]]
df[2]=[re.sub(r'.*\((.*)\)', r'\1', string) for string in df[2]]

df.replace("Not assigned", np.nan, inplace=True)
#print(df[df[2]==np.nan])
df.dropna(inplace=True)
df[2].replace(r' / ', ', ', inplace=True, regex=True)

df.columns=['PostalCode', 'Borough', 'Neighborhood']
df.reset_index(drop=True, inplace=True)
#print(df[df['Neighborhood']==np.nan])
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


###### Example: Cell with two boroughs (after transformation)
compare with the cell value before transformation

In [9]:
# Test for a cell with two boroughs:
df[df['PostalCode']=='M3K']

,PostalCode,Borough,Neighborhood
40,M3K,"North York, East","Downsview, CFB Toronto"


#### Shape of the dataframe:

In [10]:
df.shape

(103, 3)

### Task for week 3 (part 2)

#### Adding new columns:

In [11]:
df['Latitude']=np.nan
df['Longitude']=np.nan
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,NaN,NaN
1,M4A,North York,Victoria Village,NaN,NaN
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",NaN,NaN
3,M6A,North York,"Lawrence Manor, Lawrence Heights",NaN,NaN
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",NaN,NaN


#### Filling latitude and longitude data:

In [12]:
!wget -q -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data

geo_coord = pd.read_csv("C:/Users\Zharas\Documents\@ds\@coursera\9 Capstone Project\Geospatial_Coordinates.csv")
geo_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
geo_coord.columns

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')

In [14]:
geo_coord.set_index(geo_coord['Postal Code'], drop = True, inplace = True)
print(geo_coord.head())

            Postal Code   Latitude  Longitude
Postal Code                                  
M1B                 M1B  43.806686 -79.194353
M1C                 M1C  43.784535 -79.160497
M1E                 M1E  43.763573 -79.188711
M1G                 M1G  43.770992 -79.216917
M1H                 M1H  43.773136 -79.239476


In [15]:
df.set_index(df['PostalCode'], drop = True, inplace = True)

In [16]:
print(df.head())

           PostalCode           Borough  \
PostalCode                                
M3A               M3A        North York   
M4A               M4A        North York   
M5A               M5A  Downtown Toronto   
M6A               M6A        North York   
M7A               M7A  Downtown Toronto   

                                           Neighborhood  Latitude  Longitude  
PostalCode                                                                    
M3A                                           Parkwoods       NaN        NaN  
M4A                                    Victoria Village       NaN        NaN  
M5A                           Regent Park, Harbourfront       NaN        NaN  
M6A                    Lawrence Manor, Lawrence Heights       NaN        NaN  
M7A         Queen's Park, Ontario Provincial Government       NaN        NaN  


#### Adding geospacial data

In [17]:
# initialize your variable to None
lat_lng_coords = None

#print(geo_coord.head())
# loop until you get the coordinates
for zipcode in df['PostalCode']:
    #print(zipcode)
    df.loc[zipcode, 'Latitude'] = geo_coord.loc[zipcode, 'Latitude']
    df.loc[zipcode, 'Longitude'] = geo_coord.loc[zipcode, 'Longitude']

In [18]:
df.reset_index(drop = True, inplace=True)

#### The resulting dataframe is below:

In [56]:
df.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North YorkNorth,Don MillsNorth,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Task for week 3 (part 3)
#### Exploring and clustering Toronto neighborhoods

Importing necessary modules

In [22]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

I create a new dataframe "df_subset" that contains only neighborhoods within boroughs that contain the word "Toronto". 

In [20]:
df_subset = df[df['Borough'].str.contains('Toronto')].reset_index(drop=True)

In [70]:
print(df_subset['Borough'].unique())
print('\nThere are ', len(df_subset['Borough'].unique()), ' boroughs in Toronto.')
df_subset.head()


['Downtown Toronto' 'East Toronto' 'West Toronto' 'East YorkEast Toronto'
 'Central Toronto' 'Downtown TorontoStn A PO Boxes25 The Esplanade'
 'East TorontoBusiness reply mail Processing Centre969 Eastern']

There are  7  boroughs in Toronto.


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


And make sure that the dataset has all 7 boroughs and 40 neighborhoods.

In [23]:
nbor=len(df_subset['Borough'].unique())
nnei=df_subset.shape[0]
print(f'The dataframe has {nbor} boroughs and {nnei} neighborhoods.')

The dataframe has 7 boroughs and 40 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>trt_explorer</em>, as shown below.

In [24]:
address = 'Toronto'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print( 'The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
print(f'The geograpical coordinate of Toronto are {latitude}, {longitude}.')

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [25]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_subset['Latitude'], df_subset['Longitude'], 
                                           df_subset['Borough'], df_subset['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [26]:
central_data = df_subset[df_subset['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678


In [27]:
address = 'Central Toronto, Toronto'

geolocator = Nominatim(user_agent="trt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


As we did with all of Toronto, let's visualize Central Toronto and the neighborhoods in it.

In [28]:
# create map of Toronto using latitude and longitude values
map_central = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(central_data['Latitude'], central_data['Longitude'], 
                           central_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central)  
    
map_central

##### Define Foursquare Credentials and Version

In [29]:
CLIENT_ID = '3UWHZ0YNVTLMWFHA4DGJXELI4CAGONDIXQXFKAYGT0UDZ4PO' # your Foursquare ID
CLIENT_SECRET = 'IOOCFBGNARKNPWKMLAQDK253IJSCFXNWVGHR5OR5HT2SOGYJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3UWHZ0YNVTLMWFHA4DGJXELI4CAGONDIXQXFKAYGT0UDZ4PO
CLIENT_SECRET:IOOCFBGNARKNPWKMLAQDK253IJSCFXNWVGHR5OR5HT2SOGYJ


Let's explore the first neighborhood in our dataframe. Get the neighborhood's name.

In [30]:
central_data.loc[0, 'Neighborhood']

'Lawrence Park'

Get the neighborhood's latitude and longitude values.

In [31]:
neighborhood_latitude = central_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = central_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = central_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


#### Now, let's get the top 100 venues that are in Lawrence Park within a radius of 1500 meters.

In [32]:
# type your answer here
radius= 1500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=3UWHZ0YNVTLMWFHA4DGJXELI4CAGONDIXQXFKAYGT0UDZ4PO&client_secret=IOOCFBGNARKNPWKMLAQDK253IJSCFXNWVGHR5OR5HT2SOGYJ&ll=43.7280205,-79.3887901&v=20180604&radius=1500&limit=100'

Send the GET request and examine the resutls

In [33]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e8a485ddd0f85002890b906'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 63,
  'suggestedBounds': {'ne': {'lat': 43.741520513500014,
    'lng': -79.37014318158191},
   'sw': {'lat': 43.714520486499985, 'lng': -79.40743701841808}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '50e6da19e4b0d8a78a0e9794',
       'name': 'Lawrence Park Ravine',
       'location': {'address': '3055 Yonge Street',
        'crossStreet': 'Lawrence Avenue East',
        'lat': 43.72696303913755,
        'lng': -79.39438246708775,
        'labeledLatLngs': [{'label'

We know that all the information is in the *items* key. Before we proceed, let's create the **get_category_type** function.

In [34]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Zharas\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Granite Club,Gym / Fitness Center,43.733043,-79.381986
2,Sherwood Park,Park,43.716551,-79.387776
3,T-buds,Tea Room,43.731247,-79.403640
4,Bobbette & Belle,Bakery,43.731339,-79.403769


In [36]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

63 venues were returned by Foursquare.


### 2. Explore Neighborhoods in Central Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Central Toronto

In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we create a new dataframe called "central_toronto_venues"

In [38]:
# type your answer here

central_toronto_venues = getNearbyVenues(names=central_data['Neighborhood'],
                                         latitudes=central_data['Latitude'],
                                         longitudes=central_data['Longitude'])

Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
North Toronto West
The Annex, North Midtown, Yorkville
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park


In [39]:
print(central_toronto_venues.shape)
central_toronto_venues.head()

(839, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.72802,-79.38879,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.72802,-79.38879,Granite Club,43.733043,-79.381986,Gym / Fitness Center
2,Lawrence Park,43.72802,-79.38879,Sherwood Park,43.716551,-79.387776,Park
3,Lawrence Park,43.72802,-79.38879,T-buds,43.731247,-79.403640,Tea Room
4,Lawrence Park,43.72802,-79.38879,Bobbette & Belle,43.731339,-79.403769,Bakery


Let's check how many venues were returned for each neighborhood

In [40]:
central_toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,100,100,100,100,100,100
Davisville North,99,99,99,99,99,99
Forest Hill North & West,100,100,100,100,100,100
Lawrence Park,63,63,63,63,63,63
"Moore Park, Summerhill East",82,82,82,82,82,82
North Toronto West,100,100,100,100,100,100
Roselawn,95,95,95,95,95,95
"Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park",100,100,100,100,100,100
"The Annex, North Midtown, Yorkville",100,100,100,100,100,100


#### Let's find out how many unique categories can be curated from all the returned venues

In [41]:
categ = len(central_toronto_venues['Venue Category'].unique())
print(f'There are {categ} unique categories.')

There are 138 unique categories.


### 3. Analyze Each Neighborhood

In [42]:
# one hot encoding
cent_toronto_onehot = pd.get_dummies(central_toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
cent_toronto_onehot['Neighborhood'] = central_toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [cent_toronto_onehot.columns[-1]] + list(cent_toronto_onehot.columns[:-1])
cent_toronto_onehot = cent_toronto_onehot[fixed_columns]

cent_toronto_onehot.head()

,Neighborhood,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,...,Thai Restaurant,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Lawrence Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Lawrence Park,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [43]:
cent_toronto_onehot.shape

(839, 139)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [44]:
cent_toronto_grouped = cent_toronto_onehot.groupby('Neighborhood').mean().reset_index()
cent_toronto_grouped

,Neighborhood,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,...,Thai Restaurant,Toy / Game Store,Track,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Davisville,0.010000,0.000000,0.020000,0.000000,0.000000,0.010000,0.010000,0.050000,0.010000,...,0.020000,0.000000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.00,0.020000
1,Davisville North,0.010101,0.000000,0.020202,0.000000,0.000000,0.010101,0.000000,0.030303,0.010101,...,0.020202,0.000000,0.000000,0.000000,0.010101,0.010101,0.010101,0.010101,0.00,0.020202
2,Forest Hill North & West,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.030000,0.020000,0.050000,...,0.000000,0.000000,0.000000,0.040000,0.010000,0.000000,0.000000,0.000000,0.00,0.000000
3,Lawrence Park,0.000000,0.000000,0.031746,0.000000,0.015873,0.000000,0.000000,0.047619,0.031746,...,0.000000,0.015873,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,"Moore Park, Summerhill East",0.036585,0.000000,0.000000,0.012195,0.012195,0.000000,0.012195,0.024390,0.012195,...,0.012195,0.000000,0.000000,0.036585,0.012195,0.000000,0.012195,0.000000,0.00,0.012195
5,North Toronto West,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.010000,...,0.000000,0.000000,0.010000,0.000000,0.010000,0.010000,0.010000,0.000000,0.00,0.020000
6,Roselawn,0.000000,0.010526,0.000000,0.000000,0.000000,0.000000,0.021053,0.021053,0.031579,...,0.000000,0.000000,0.010526,0.021053,0.000000,0.000000,0.010526,0.000000,0.00,0.010526
7,"Summerhill West, Rathnelly, South Hill, Forest...",0.030000,0.000000,0.000000,0.010000,0.020000,0.000000,0.020000,0.020000,0.020000,...,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.010000,0.000000,0.00,0.010000
8,"The Annex, North Midtown, Yorkville",0.010000,0.010000,0.000000,0.000000,0.020000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.000000,0.01,0.000000


Let's confirm the new size

In [45]:
cent_toronto_grouped.shape

(9, 139)

#### Let's print each neighborhood along with the top 5 most common venues

In [46]:
num_top_venues = 5

for hood in cent_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = cent_toronto_grouped[cent_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.06
2              Bakery  0.05
3                Park  0.04
4         Pizza Place  0.04


----Davisville North----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.08
2                Park  0.04
3         Pizza Place  0.03
4                Café  0.03


----Forest Hill North & West----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.07
2                Bank  0.05
3    Sushi Restaurant  0.05
4                Café  0.04


----Lawrence Park----
                venue  freq
0         Coffee Shop  0.08
1    Sushi Restaurant  0.06
2                Café  0.05
3  Italian Restaurant  0.05
4              Bakery  0.05


----Moore Park, Summerhill East----
                 venue  freq
0   Italian Restaurant  0.07
1                 Park  0.07
2          Coffee Shop  0.06
3     Sushi Restaurant  0.05
4  American Restaurant  0.04


----North

#### Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [47]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [48]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = cent_toronto_grouped['Neighborhood']

for ind in np.arange(cent_toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(cent_toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Coffee Shop,Italian Restaurant,Bakery,Park,Pizza Place,Indian Restaurant,Gym,Café,Japanese Restaurant,Restaurant
1,Davisville North,Italian Restaurant,Coffee Shop,Park,Sushi Restaurant,Restaurant,Gym,Bakery,Indian Restaurant,Pizza Place,Café
2,Forest Hill North & West,Italian Restaurant,Coffee Shop,Sushi Restaurant,Bank,Trail,Café,Japanese Restaurant,Bagel Shop,Liquor Store,Middle Eastern Restaurant
3,Lawrence Park,Coffee Shop,Sushi Restaurant,Bakery,Café,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Bank,Park,Pub
4,"Moore Park, Summerhill East",Italian Restaurant,Park,Coffee Shop,Sushi Restaurant,American Restaurant,Restaurant,Trail,Grocery Store,Liquor Store,Kids Store


### 4. Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 4 clusters.

In [49]:
# set number of clusters
kclusters = 3

cent_toronto_grouped_clustering = cent_toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(cent_toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 2, 0, 2, 2, 0, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [50]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

cent_toronto_merged = central_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
cent_toronto_merged = cent_toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

cent_toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Coffee Shop,Sushi Restaurant,Bakery,Café,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Bank,Park,Pub
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Skating Rink,Pharmacy,Diner,Café,Bank,Trail
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Italian Restaurant,Coffee Shop,Park,Sushi Restaurant,Restaurant,Gym,Bakery,Indian Restaurant,Pizza Place,Café
3,M5P,Central Toronto,Forest Hill North & West,43.696948,-79.411307,2,Italian Restaurant,Coffee Shop,Sushi Restaurant,Bank,Trail,Café,Japanese Restaurant,Bagel Shop,Liquor Store,Middle Eastern Restaurant
4,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Bakery,Fast Food Restaurant,Japanese Restaurant,Breakfast Spot,Bookstore,Mexican Restaurant


#### Finally, let's visualize the resulting clusters

In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(cent_toronto_merged['Latitude'], 
                                  cent_toronto_merged['Longitude'],
                                  cent_toronto_merged['Neighborhood'], 
                                  cent_toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 5. Examine Clusters

Here we examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, a name is assigned to each cluster.

#### 1. Park zone

In [52]:
cent_toronto_merged.loc[cent_toronto_merged['Cluster Labels'] == 0, cent_toronto_merged.columns[[1] + list(range(5, cent_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,0,Italian Restaurant,Park,Coffee Shop,Sushi Restaurant,American Restaurant,Restaurant,Trail,Grocery Store,Liquor Store,Kids Store
8,Central Toronto,0,Italian Restaurant,Café,Park,Coffee Shop,Sushi Restaurant,American Restaurant,Burger Joint,Liquor Store,Pizza Place,Modern European Restaurant


#### 2. European restaurants

In [53]:
cent_toronto_merged.loc[cent_toronto_merged['Cluster Labels'] == 1, cent_toronto_merged.columns[[1] + list(range(5, cent_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Central Toronto,1,Italian Restaurant,Coffee Shop,Park,Sushi Restaurant,Restaurant,Gym,Bakery,Indian Restaurant,Pizza Place,Café
5,Central Toronto,1,Coffee Shop,Restaurant,Café,Italian Restaurant,Grocery Store,Korean Restaurant,Park,Japanese Restaurant,Bakery,Bookstore
6,Central Toronto,1,Coffee Shop,Italian Restaurant,Bakery,Park,Pizza Place,Indian Restaurant,Gym,Café,Japanese Restaurant,Restaurant


#### 3. Japanese Restaurants 

In [54]:
cent_toronto_merged.loc[cent_toronto_merged['Cluster Labels'] == 2, cent_toronto_merged.columns[[1] + list(range(5, cent_toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,2,Coffee Shop,Sushi Restaurant,Bakery,Café,Italian Restaurant,Sandwich Place,Fast Food Restaurant,Bank,Park,Pub
1,Central Toronto,2,Coffee Shop,Italian Restaurant,Sushi Restaurant,Pizza Place,Skating Rink,Pharmacy,Diner,Café,Bank,Trail
3,Central Toronto,2,Italian Restaurant,Coffee Shop,Sushi Restaurant,Bank,Trail,Café,Japanese Restaurant,Bagel Shop,Liquor Store,Middle Eastern Restaurant
4,Central Toronto,2,Coffee Shop,Italian Restaurant,Café,Sushi Restaurant,Bakery,Fast Food Restaurant,Japanese Restaurant,Breakfast Spot,Bookstore,Mexican Restaurant
